In [1]:
!pip install -r requirements.txt

In [15]:
import bash_runner as bash
import importlib
from pathlib import Path
importlib.reload(bash)

<module 'bash_runner' from '/home/jupyter/data-science-examples/tutorials/script-languages/bash_runner.py'>

In [69]:
slc_path="script-languages-release"
if not Path("script-languages-release").exists():
    bash.run("""
    git clone https://github.com/exasol/script-languages-release --recursive
    """)
else:
    bash.run(f"""
    cd {slc_path}
    git reset --hard origin/master 
    git submodule foreach git reset --hard origin/master 
    """)
    


HEAD is now at 3bb2b47 Renaming certain things to udf plugins (#171)
Entering 'script-languages'
HEAD is now at ed13033 Renaming certain things to udf plugins (#168)


In [24]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python3-ds-EXASOL-6.1.0 --export-path containers
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages
Using following pipenv executable: /opt/conda/bin/pipenv
Using existing virtual environment
Installing dependencies
Virtualenv already exists!
Removing existing virtualenv...
Creating a virtualenv for this project...
Pipfile: /home/jupyter/data-science-examples/tutorials/script-languages/script-languages-release/script-languages/Pipfile
Using /opt/conda/bin/python3.7m (3.7.9) to create virtualenv...
⠼ Creating virtual environment...created virtual environment CPython3.7.9.final.0-64 in 205ms
  creator CPython3Posix(dest=/home/jupyter/.local/share/virtualenvs/script-languages-VP1Xj6ma, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/home/jupyter/.local/share/virtualenv)
    added seed packages: pip==20.3.1, setuptools==51.0.0, wheel==0.36.2
  activators Bash

In [25]:
bash.run(f"""tail {slc_path}/exaslct.log""")

===== Luigi Execution Summary =====

The command took 5.551279 s

Cached container under /home/jupyter/data-science-examples/tutorials/script-languages/script-languages-release/.build_output/cache/exports/python3-ds-EXASOL-6.1.0-release-EYFRS54NWXPTDZOBU2ZWGIXIID77PTCFMM2LLHPQNWDGQAQUNIAA.tar.gz

Copied container to containers/python3-ds-EXASOL-6.1.0_release.tar.gz




In [40]:
bash.run(f"""
find {slc_path}/.build_output/jobs/*/outputs -type f
""")

script-languages-release/.build_output/jobs/2021_02_02_11_00_53_ExportContainers/outputs/ExportContainers_b032926fcd/ExportFlavorContainer_8eba5879f8/ExportContainerTask_a69810acba/logs/extract_release_file.log
script-languages-release/.build_output/jobs/2021_02_02_11_00_53_ExportContainers/outputs/ExportContainers_b032926fcd/ExportFlavorContainer_8eba5879f8/ExportContainerTask_a69810acba/logs/pack_release_file.log
script-languages-release/.build_output/jobs/2021_02_02_11_00_53_ExportContainers/outputs/ExportContainers_b032926fcd/ExportFlavorContainer_8eba5879f8/DockerCreateImageTask_f2e68ce3b8/DockerPullImageTask_f2e68ce3b8/logs/pull_docker_db_image.log
script-languages-release/.build_output/jobs/2021_02_02_11_00_53_ExportContainers/outputs/ExportContainers_b032926fcd/command_line_output
script-languages-release/.build_output/jobs/2021_02_02_11_15_40_ExportContainers/outputs/ExportContainers_b73ac2ac38/command_line_output


In [38]:
bash.run(f"""
find {slc_path}/flavors/  -maxdepth 1
""")

script-languages-release/flavors/
script-languages-release/flavors/fancyr-EXASOL-6.1.0
script-languages-release/flavors/standard-EXASOL-6.1.0
script-languages-release/flavors/python3-ds-EXASOL-6.1.0
script-languages-release/flavors/docs
script-languages-release/flavors/standard-EXASOL-7.0.0
script-languages-release/flavors/README.md
script-languages-release/flavors/standard-EXASOL-6.2.0
script-languages-release/flavors/FLAVORS_DOCUMENTATION.md
script-languages-release/flavors/python3-ds-cuda-preview-EXASOL-6.1.0


In [48]:
bash.run(f""" 
find -L {slc_path}/flavors/python3-ds-EXASOL-6.1.0 -maxdepth 2
""")

script-languages-release/flavors/python3-ds-EXASOL-6.1.0
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/Dockerfile
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/base_test_build_run
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/release
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/testconfig
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/flavor_test_build_run
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/base_test_deps
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/language_definition
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/build_run
script-languages-release/flavor

In [49]:
bash.run(f""" 
find -L {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization -type f
""")

script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/Dockerfile
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/apt_get_packages


In [50]:
bash.run(f""" 
cat {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/Dockerfile
""")

############################################################################################
############################################################################################
# This Dockerfile allows you to extend this flavor by installing packages or adding files. 
# IF you didn't change the lines below, you can add packages and their version to the  
# files in ./packages and they get automatically installed.                                
############################################################################################
############################################################################################

#######################################################################
#######################################################################
# Do not change the following lines unless you know what you are doing 
#######################################################################
###################################################################

In [70]:
bash.run(f""" 
cat {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
""")

# This file specifies the package list which gets installed via pip for python3.
# You must specify the the package and its version separated by a |.
# We recommend here the usage of package versions, to ensure that the container 
# builds are reproducible. However, we allow also packages without version.
# As you can see, this file can contain comments which start with #.
# If a line starts with # the whole line is a comment, however you can
# also start a comment after the package definition.

#tensorflow-probability|0.9.0


In [71]:
bash.run(f""" 
echo "dask[complete]|2021.1.1 " >> {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
""")

In [72]:
bash.run(f""" 
cat {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
""")

# This file specifies the package list which gets installed via pip for python3.
# You must specify the the package and its version separated by a |.
# We recommend here the usage of package versions, to ensure that the container 
# builds are reproducible. However, we allow also packages without version.
# As you can see, this file can contain comments which start with #.
# If a line starts with # the whole line is a comment, however you can
# also start a comment after the package definition.

#tensorflow-probability|0.9.0
dask[complete]|2021.1.1 


In [76]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python3-ds-EXASOL-6.1.0 --export-path containers | grep -E "DockerPullImageTask|DockerBuildImageTask"
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages
Virtualenv already exists!
Removing existing virtualenv...
Creating a virtualenv for this project...
Pipfile: /home/jupyter/data-science-examples/tutorials/script-languages/script-languages-release/script-languages/Pipfile
Using /opt/conda/bin/python3.7m (3.7.9) to create virtualenv...
⠼ Creating virtual environment...created virtual environment CPython3.7.9.final.0-64 in 195ms
  creator CPython3Posix(dest=/home/jupyter/.local/share/virtualenvs/script-languages-VP1Xj6ma, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/home/jupyter/.local/share/virtualenv)
    added seed packages: pip==20.3.1, setuptools==51.0.0, wheel==0.36.2
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator

✔ Successfully created v